In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open("sherlock-holm.es_stories_plain-text_advs.txt", 'r', encoding='utf-8') as file:
    text = file.read()
    tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [2]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 133s 43ms/step - loss: 6.2425 - accuracy: 0.0757
Epoch 2/100
3010/3010 [==============================] - 106s 35ms/step - loss: 5.5049 - accuracy: 0.1261
Epoch 3/100
3010/3010 [==============================] - 106s 35ms/step - loss: 5.1189 - accuracy: 0.1501
Epoch 4/100
3010/3010 [==============================] - 99s 33ms/step - loss: 4.7944 - accuracy: 0.1677
Epoch 5/100
3010/3010 [==============================] - 100s 33ms/step - loss: 4.4924 - accuracy: 0.1852
Epoch 6/100
3010/3010 [==============================] - 101s 34ms/step - loss: 4.2104 - accuracy: 0.2026
Epoch 7/100
3010/3010 [==============================] - 101s 34ms/step - loss: 3.9399 - accuracy: 0.2284
Epoch 8/100
3010/3010 [==============================] - 102s 34ms/step - loss: 3.6830 - accuracy: 0.2586
Epoch 9/100
3010/3010 [==============================] - 102s 34ms/step - loss: 3.4398 - accuracy: 0.2916
Epoch 10/100
3010/3010 [=======================

3010/3010 [==============================] - 100s 33ms/step - loss: 0.5438 - accuracy: 0.8615
Epoch 78/100
3010/3010 [==============================] - 96s 32ms/step - loss: 0.5408 - accuracy: 0.8620
Epoch 79/100
3010/3010 [==============================] - 97s 32ms/step - loss: 0.5378 - accuracy: 0.8625
Epoch 80/100
3010/3010 [==============================] - 101s 34ms/step - loss: 0.5332 - accuracy: 0.8640
Epoch 81/100
3010/3010 [==============================] - 103s 34ms/step - loss: 0.5374 - accuracy: 0.8623
Epoch 82/100
3010/3010 [==============================] - 101s 34ms/step - loss: 0.5314 - accuracy: 0.8639
Epoch 83/100
3010/3010 [==============================] - 104s 35ms/step - loss: 0.5336 - accuracy: 0.8624
Epoch 84/100
3010/3010 [==============================] - 104s 35ms/step - loss: 0.5288 - accuracy: 0.8636
Epoch 85/100
3010/3010 [==============================] - 105s 35ms/step - loss: 0.5301 - accuracy: 0.8634
Epoch 86/100
3010/3010 [============================

In [7]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 27ms/step
I will leave if they have been your
